In [108]:
# 모두닥 크롤링 모듈

In [109]:
import sys

sys.version

'3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]'

In [110]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [111]:
import pandas as pd    # csv
from selenium.webdriver.chrome.options import *
from bs4 import *
import time
import re
import base64    # encoder
# webdriver = 웹페이지를 제어하기 위한 모듈 >> 웹 그자체가 됨!
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [112]:
# 크롤링 하려는 병원 이동
def move_to_hospital(url):
    driver.get(url)
    time.sleep(1)

    
# 병원의 리뷰 페이지 수 계산
def count_page():
    num_review = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[2]/div/div[1]/div[1]').text[7:]
    num_review = int(num_review)

    num_page = (num_review // 10) + 1

    return num_page


# 텍스트 전처리
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]' , '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', cleaned_text)
    cleaned_text = re.sub('[\n]', ' ', cleaned_text)
    
    return cleaned_text



In [113]:
# 모두닥 페이지 url
modoodoc_url = "https://www.modoodoc.com/"

# # 피부과 페이지 url
# url_derma = "https://www.modoodoc.com/hospitals/?search_query=%ED%94%BC%EB%B6%80%EA%B3%BC"

## 크롬 드라이버 실행
#  wb.Chrome() = 컴퓨터가 크롬창을 씀
driver = wb.Chrome()

# 모두닥 페이지 이동
driver.get(modoodoc_url)

# 로그인
id = 'random1234@random.com'
password = 'random1234'

driver.find_element('xpath', '/html/body/nav/div/div/ul[2]/ul/li[3]/a').click() #id 속성으로 접근

user_email = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[1]')
user_email.send_keys(id)
user_password = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[2]')
user_password.send_keys(password)

# 로그인 버튼 클릭
driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/button').click()
time.sleep(3)


In [114]:
# 크롤링 하려는 병원 이동
hospital_url = "https://www.modoodoc.com/hospital/4470/%ED%95%9C%EA%B0%95%EC%88%98%EB%B3%91%EC%9B%90-%EC%84%9C%EC%9A%B8-%EC%96%91%ED%8F%89%EB%8F%991%EA%B0%80/"
move_to_hospital(hospital_url)

In [115]:
# 크롤링 후 리스트 삽입

data = []
data_unit = []

current_url = driver.current_url
num_page = count_page()




for page in range(1, num_page+1):
    url = current_url + f"?page={page}"
    driver.get(url)
    time.sleep(1)
    
    for i in range(2, 16):
        try:
            # 병원 이름 크롤링
            hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

            # 받은 치료 이름 크롤링
            treatment = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{i}]/div/div[2]/div[2]').text


            # 받은 치료 명시 안된 부분 처리
            if treatment[0] != '받':
                continue
            treatment = treatment[8:]

            # 리뷰 내용 크롤링
            review = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{i}]/div/div[2]/div[3]').text
            # 리뷰 텍스트 전처리
            review = clean_text(review)

            # 크롤링 데이터 리스트 삽입
            data_unit = [hospital, treatment, review]
            data.append(data_unit)

        except:
            pass

In [116]:
# 데이터 프레임 생성 및 데이터 입력
df = pd.DataFrame(columns = ['병원 이름', '받은 치료', '리뷰'])

num = len(data)

for i in range(num):
    df.loc[i]=data[i]
   
df
# # 엑셀 저장
# df.to_excel('1234.xlsx')

,병원 이름,받은 치료,리뷰
0,한강수병원,화상치료,가장 친절하신 선생님이심 야간 진료가 있다는건 장점 한분께 진료를 계속 볼 수 없...
1,한강수병원,화상치료,화상은 화상 전문병원으로 가야한다고 해서 김포에서 찾아갔는데 생각보다 자주 병원에 ...
2,한강수병원,화상치료,늦은 저녁에 끓는 기름이 담긴 프라이팬을 팔로쳐서 양손에 화상을 입었습니다 문여는 ...
3,한강수병원,화상치료,늦은 저녁에 끓는 기름이 담긴 프라이팬을 팔로쳐서 양손에 화상을 입었습니다 문여는 ...
4,한강수병원,화상치료,손가락 화상을 당해서 화상 전문 병원을 찾았습니다 결론부터 이야기 하자면 이 병원은...
5,한강수병원,화상치료,팔에 화상 입어서 방문했습니다 아픈거보다 흉터 생각에 너무 힘들고 동네 병원에서 드...
6,한강수병원,화상치료,다리에 화상을 입어서 화상 전문 병원을 찾던 중 회사 근처 유명한 병원이 있다고 해...
7,한강수병원,화상치료,포트 팔팔 끓는물을 오른손에 부어서 오른손 전체에 수포 가득한 화상을 입었음ㅜ 손에...
8,한강수병원,화상치료,다친상처로 가게되었는데 처음갔을때도 그렇고 계속해서 친절하시구 잘치료해주십니다 약간...
9,한강수병원,피지낭종(표피낭종)제거,귀 염증땜에 갔었는데요 보시더니 별로 심하지 않고 그냥 두는 게 나을 것 같다고 하...


In [117]:
# 버전 체크

print(webdriver.__version__)

4.3.0


In [122]:
# 엑셀 저장
df.to_excel('prototype.xlsx')

In [118]:
# # 병원 이름 크롤링
# hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

# # 받은 치료 이름 크롤링
# treatment = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[2]').text[8:]

# # 리뷰 내용 크롤링
# review = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[3]').text

# # 크롤링 데이터 리스트 삽입
# data_unit = [hospital, treatment, review]
# data.append(data_unit)
# data

In [119]:
# parsing을 위한 함수를 생성

In [120]:
# def remove_html_tags(data):
#     p = re.compile(r'<.*?>')
#     return p.sub(' ', str(data))

# def get_crawl(URL):
#     response = driver.get(URL)
#     html = driver.page_source
#     soup7 = BeautifulSoup(html, 'html.parser')
#     ex_id_divs = soup7.find('div', {'id': 'view_content'})
#     crawl_data = remove_html_tags(ex_id_divs)
#     return crawl_data

In [121]:
# # 리뷰 페이지 이동
# num_page = count_page()

# current_url = driver.current_url

# for i in range(2, num_page+1):
#     url = current_url + f"?page={i}"
#     driver.get(url)
#     time.sleep(1)